# Example 8.3 (from the book)

In [1]:
import numpy as np
from cvxopt import matrix, solvers

The input data is given as follows:

In [2]:
X = np.array([[0,2,2,3],[0,2,0,0]]).T
y = np.array([-1,-1,1,1])
print('Dimension of X is ', X.shape)
print('Dimension of y is ', y.shape)

Dimension of X is  (4, 2)
Dimension of y is  (4,)


## Primal Problem Formulation
$\newcommand{\R}{\mathbb{R}}$
We start witht he primal problem formulation. According to the equation (8.6) on page 9 in the book we have the following:
$$
min_{u \in \R^L} \ \ \frac{1}{2}\mathbf u^TQ\mathbf u + \mathbf p^T\mathbf u
$$
$$
\text{s.t. }\ \ A\mathbf u \geq c
$$

Now, we need to define the input for the QP solver for:  
- $Q$ with $(L,L)$ dimension  
- $\mathbf p$ with $(L,1)$ dimension 
- $A$ with $(M,L)$ dimension
- $\mathbf c$ with $(M,1)$ dimension

So, let's get the dimensions first:

In [3]:
d = X.shape[1]
L = d + 1
M = X.shape[0]
print('L = ',L,'and M = ',M)

L =  3 and M =  4


We will use `cvxopt` solver with the structure as `cvxopt.solvers.qp(P, q[, G, h[, A, b[, solver[, initvals]]]])` which solves the following:
$$
min \frac{1}{2}x^TPx+q^Tx
$$
$$
\text{s.t }Gx \preceq h
$$
$$
Ax=b
$$

Note that `P` and `q` are required whereas other inputs are optional. To equate it with our example we have the following:  
- $\mathbf x \to \mathbf u$
- $P \to Q$  
- $q \to p$
- $G \to A$
- $c \to h$  

Note that we don't have $Ax=b$ condition, so we neglect this part. Also we need to negate $Au \geq c$ to obtain the correct structure for the solver since it requires $Gx \preceq h$. Now we can define each of the needed inputs for the QP:

In [4]:
p = np.zeros((L,1))
Q = np.vstack([p.T,np.column_stack([p[1:],np.eye(d)])])
A = -np.column_stack([y,y[:,None]*X])
c = -np.ones((M))

In [5]:
P = matrix(Q, tc = 'd')
q = matrix(p, tc = 'd')
G = matrix(A, tc = 'd')
h = matrix(c, tc = 'd')

u = solvers.qp(P, q, G, h)['x']
np.array(u)

     pcost       dcost       gap    pres   dres
 0:  3.2653e-01  1.9592e+00  6e+00  2e+00  4e+00
 1:  1.5796e+00  8.5663e-01  7e-01  1e-16  2e-15
 2:  1.0195e+00  9.9227e-01  3e-02  0e+00  2e-15
 3:  1.0002e+00  9.9992e-01  3e-04  3e-16  1e-15
 4:  1.0000e+00  1.0000e+00  3e-06  3e-16  8e-16
 5:  1.0000e+00  1.0000e+00  3e-08  0e+00  1e-15
Optimal solution found.


array([[-1.00000001],
       [ 1.00000001],
       [-1.00000001]])

Since $\mathbf u = [b ,\mathbf w]$ we have that $b=-1$, $w_1=1$ and $w_2=-1$, which coresponds to the solution given in the book. We can also do the same computation using the SVC function with linear kernel and $C=\infty$.

## Dual Problem Formulation
Now we can compute the dual formulation of the same example to illustrate the process. Note that now we have the following setup:
$$
min_{\alpha \in \R^N} \ \ \frac{1}{2}\sum_{m=1}^{N}\sum_{n=1}^{N} y_ny_m\alpha_n\alpha_m\mathbf x_n^T\mathbf x_m - \sum_{n=1}^{N} \alpha_n
$$
$$
\text{subject to:  }\ \ \sum_{n=1}^{N} y_n\alpha_n= 0 
$$
$$
\alpha_n \geq 0
$$
or
$$
min_{\alpha \in \R^N} \ \ \frac{1}{2}\alpha^TQ_D\alpha-1_N^T\alpha
$$
$$
\text{subject to:  }\ \ A_D\alpha \geq 0_{N+2}
$$

where
$$
Q_D=
\begin{bmatrix}
y_1y_1\mathbf x_1^T\mathbf x_1&\cdots&y_1y_N\mathbf x_1^T\mathbf x_N\\
y_2y_1\mathbf x_2^T\mathbf x_1&\cdots&y_2y_N\mathbf x_2^T\mathbf x_N\\
\vdots&\vdots&\vdots \\
y_Ny_1\mathbf x_N^T\mathbf x_1&\cdots&y_Ny_N\mathbf x_N^T\mathbf x_N\\
\end{bmatrix}
$$
and
$$
A_D=
\begin{bmatrix}
\mathbf y^T\\
-\mathbf y^T\\
I_{N\times N}\\
\end{bmatrix}
$$

So we have another QP problem for which we can use the same solver as in previous section, but first we need to define the inputs correctly.

In [6]:
y = y.reshape(-1,1)
QD = (np.dot(y,y.T)) * (np.dot(X,X.T))
P = matrix(QD, tc = 'd')

In [7]:
AD = -np.vstack([y.T,-y.T,np.eye(M)])
A = matrix (AD, tc = 'd')

In [8]:
q = matrix(-np.ones((M,1)), tc = 'd')
h = matrix(np.zeros((M+2)), tc='d')

In [9]:
sol = solvers.qp(P,q,A,h)

     pcost       dcost       gap    pres   dres
 0: -1.0249e+00 -2.3063e+00  1e+01  3e+00  2e+00
 1: -8.2851e-01 -1.7889e+00  1e+00  1e-01  6e-02
 2: -9.8330e-01 -1.0720e+00  1e-01  4e-03  2e-03
 3: -9.9979e-01 -1.0008e+00  1e-03  4e-05  2e-05
 4: -1.0000e+00 -1.0000e+00  1e-05  4e-07  2e-07
 5: -1.0000e+00 -1.0000e+00  1e-07  4e-09  2e-09
Optimal solution found.


In [10]:
alpha = np.around(np.array(sol['x']), decimals = 2)
alpha

array([[ 0.5],
       [ 0.5],
       [ 1. ],
       [ 0. ]])

Count number of support vectors

In [11]:
len(alpha[alpha>0])

3

From the obtained $\alpha$ we can now get the weights using $\mathbf w^*=\sum_{n=1}^{N} y_n\alpha_n\mathbf x_n$.

In [12]:
w_svm1 = np.dot(X.T,y*alpha)
w_svm1

array([[ 1.],
       [-1.]])

Now to get $b$ we need $\alpha > 0$ in order for the constraint to be exactly satisfied and hence we can solve for $b$. $\alpha_1 = \frac{1}{2}>0$ then
$y_n(\mathbf w^{*T}x_1+b^*)=1$. So first we need $y$ and $X$ for which $\alpha > 0$.

In [13]:
ys = y[alpha>0]
idx = alpha.reshape(X.shape[0])
Xs = X[idx > 0,:]

We can pick any point from `Xs` to obtain $b$.

In [14]:
i = 1
b = ys[i] - np.sum(w_svm1.T*Xs[i])
b

-1.0

Hence, our final weights are $[b,\mathbf w] = [-1,1,-1]$.